In [5]:
import scipy.io
import numpy as np
from numpy import linalg as LA
import time
from PIL import Image

Function to resize (2576,1) image vector into a (46,56) size image and display it.

In [6]:
def show_img(img):
    temp = img.copy()
    temp.resize((46,56))
    im = Image.fromarray(temp.T)
    im.show()

Load face data 

In [7]:
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

Segmenting data into training and testing data. A 8/2 ratio has been used

In [9]:
D,N = raw_data.shape
raw_data = np.transpose(raw_data)

training_data = np.empty([int(N*0.8), D])
testing_data = np.empty([int(N*0.2), D])

for x in range(N/10):
    training_data[x*8:(x+1)*8] = raw_data[x*10:x*10+8]
    testing_data[x*2:(x+1)*2] = raw_data[x*10+8:(x+1)*10]

raw_data = np.transpose(raw_data)
training_data = np.transpose(training_data)
testing_data = np.transpose(testing_data)

2576


Calculate the mean face from the training dataset

In [243]:
mean_face = training_data.mean(axis=1).reshape(-1,1)
show_img(mean_face)

In [244]:
A = training_data - mean_face
S = (1/N)*np.dot(np.transpose(A),A)
w, v = LA.eig(S)
v /= LA.norm(v,ord=2,axis=0)
u = np.dot(A,v)
u /= LA.norm(u,ord=2,axis=0)
id = np.argsort(np.abs(w))[::-1]
w = w[id]
u = np.real(u[:,id])
u_norm = u



(416, 416)


In [245]:
# for i in range(10):
#     asdf = u_norm[:,i].reshape(46,56)
#     im = Image.fromarray(asdf)
#     im.show()

In [246]:
input = testing_data
delta = input - mean_face

Compute weights for testing images

In [247]:
n_face = 100
eigface = u_norm[:,:n_face]
weights = np.matmul(delta.T, eigface)
print(eigface.shape)
print(delta.shape)
print(weights.shape)

(2576, 100)
(2576, 104)
(104, 100)


In [248]:
reconstructed_faces = np.matmul(weights, eigface.T)
print(reconstructed_faces[0])
print(reconstructed_faces.shape)

face = []
for i in range(len(reconstructed_faces)):
    face.append(reconstructed_faces[i][:] + mean_face.T)


for x in face[:20]:
    show_img(x)

[-13.88510814 -16.47668337 -11.93320382 ...  34.87826407  37.63854347
  42.84121952]
(104, 2576)
